In [1]:
import dash 
import plotly.express as px
import plotly.graph_objects as go
from dash import html, dcc ,State
from dash.dependencies import Input, Output

In [2]:
import requests
import io
import pandas as pd

# URL of the CSV file
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"

# Fetch the data from the URL
response = requests.get(URL)

# Raise an error if the request failed
response.raise_for_status()

# Convert the response content into a readable format for pandas
csv_content = io.StringIO(response.text)

# Read the CSV data into a pandas dataframe
df = pd.read_csv(csv_content)

# Print confirmation
print('Data downloaded and read into a dataframe!')

# Optional: Show the first few rows of the dataframe
print(df.head())

Data downloaded and read into a dataframe!
        Date  Year Month  Recession  Consumer_Confidence  Seasonality_Weight  \
0  1/31/1980  1980   Jan          1               108.24                0.50   
1  2/29/1980  1980   Feb          1                98.75                0.75   
2  3/31/1980  1980   Mar          1               107.48                0.20   
3  4/30/1980  1980   Apr          1               115.01                1.00   
4  5/31/1980  1980   May          1                98.72                0.20   

       Price  Advertising_Expenditure  Competition     GDP  Growth_Rate  \
0  27483.571                     1558            7  60.223     0.010000   
1  24308.678                     3048            4  45.986    -0.309594   
2  28238.443                     3137            3  35.141    -0.308614   
3  32615.149                     1653            7  45.673     0.230596   
4  23829.233                     1319            4  52.997     0.138197   

   unemployment_rate  Aut

In [3]:
app=dash.Dash(__name__)
app.layout=html.Div(children=[
    html.H1("Automobile Sales Statistics Dashboard",style={"text-align":"center", "color":'#503D36', "font-size":24}),
    html.Div(dcc.Dropdown(
        id="dropdown-statistics",
        options=[
            {"label":"Yearly Statistics", "value":"Yearly Statistics"},
            {"label":'Recession Period Statistics', "value":"Recession Period Statistics"}
        ],
        placeholder="Select a report type",
        value="Select Statistics",
        style={"width":"80%", "padding":"3px", "text-align-last":"center"}
    )),
    html.Div(dcc.Dropdown(id='select-year', 
            options=[{'label': i, 'value': i} for i in df["Year"].unique()],
            placeholder='select-year',
            value="select-year",
            style={"width":"80%", "padding":"3px", "text-align-last":"center"})),
    html.Div([
         html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),
])
])

In [4]:
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value')  
)
def update_input_container(select):
    if select=='Yearly Statistics': 
        return False
    else: 
        return True

In [5]:
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [
        Input(component_id='dropdown-statistics', component_property='value'), 
        Input(component_id='select-year', component_property='value')
    ]   
)
def update_output_container(select,year):
    if select== 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = df[df['Recession'] == 1]
        #Plot 1 Automobile sales fluctuate over Recession Period (year wise) using line chart
        # grouping data for plotting
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        # Plotting the line graph
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                x='Year',
                y='Automobile_Sales',
                title="Automobile sales fluctuate over Recession Period"
                )
            )
        #Plot 2 Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart
        # use groupby to create relevant data for plotting. 
        #Hint:Use Vehicle_Type and Automobile_Sales columns
        
        average_sales = recession_data.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index()                
        R_chart2  = dcc.Graph(
            figure=px.bar(average_sales,
            x='Vehicle_Type',
            y='Automobile_Sales',
            title="The average number of vehicles sold by vehicle type"))
        
        # Plot 3 : Pie chart for total expenditure share by vehicle type during recessions
            # grouping data for plotting
            # Hint:Use Vehicle_Type and Advertising_Expenditure columns
            
        exp_rec= recession_data.groupby("Vehicle_Type")["Advertising_Expenditure"].sum().reset_index()
        R_chart3 = dcc.Graph(
                figure=px.pie(exp_rec,
                values='Advertising_Expenditure',
                names='Vehicle_Type',
                title=" Total expenditure share by vehicle type"
            )
        )
        
        # Plot 4 Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
       #grouping data for plotting
       # Hint:Use unemployment_rate,Vehicle_Type and Automobile_Sales columns
       
        unemp_data= recession_data.groupby(['Vehicle_Type','unemployment_rate'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data,
        x='Vehicle_Type',
        y='Automobile_Sales',
        color='unemployment_rate',
        labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
        title='Effect of Unemployment Rate on Vehicle Type and Sales'))
        
        return [
            html.Div(className='chart-grid', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)]),
            html.Div(className='chart-grid', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)])
            ]

    else:
        if year:
            data_year=df[df["Year"]==year]
            
            # Plot 1 :Yearly Automobile sales using line chart for the whole period.
            # grouping data for plotting.
            # Hint:Use the columns Year and Automobile_Sales.
            
            yas= df.groupby('Year')['Automobile_Sales'].mean().reset_index()
            Y_chart1 = dcc.Graph(figure=px.line(yas,
                                                x="Year",
                                                y="Automobile_Sales",
                                                title="Yearly Automobile sales"))
            
            # Plot 2 :Total Monthly Automobile sales using line chart.
             # grouping data for plotting.
            # Hint:Use the columns Month and Automobile_Sales.
            mas=df.groupby('Month')["Automobile_Sales"].mean().reset_index()
            Y_chart2 = dcc.Graph(figure=px.line(mas,
               x='Month',
               y='Automobile_Sales',
               title='Total Monthly Automobile Sales'))
            
            # Plot bar chart for average number of vehicles sold during the given year
             # grouping data for plotting.
             # Hint:Use the columns Year and Automobile_Sales
            avr_vdata=data_year.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index()
            Y_chart3 = dcc.Graph(figure=px.bar(avr_vdata,
                                               x="Vehicle_Type", 
                                               y="Automobile_Sales" ,
                                               title='Average Vehicles Sold by Vehicle Type in the year {}'.format(year)))
            
            # Plot 4 Total Advertisement Expenditure for each vehicle using pie chart
              # grouping data for plotting.
              # Hint:Use the columns Vehicle_Type and Advertising_Expenditure
            exp_data=data_year.groupby('Vehicle_Type')["Advertising_Expenditure"].sum().reset_index()
            Y_chart4 = dcc.Graph(
                figure=px.pie(exp_data, 
                values='Advertising_Expenditure',
                names='Vehicle_Type',
                title='Total Advertisment Expenditure for Each Vehicle'))
            
            return [
                    html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)]),
                    html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)])
                    ]


In [6]:
if __name__=="__main__":
   app.run_server(debug=True)